In [1]:
# !pip install ipykernel pandas Jinja2 matplotlib

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
sys.path.append(os.path.dirname(os.getcwd()))
from aggregate_logs_to_csv import aggregate_models_to_csv, MODEL_OVERRIDES

<Figure size 640x480 with 0 Axes>

In [3]:
if not os.path.exists("../_temp"):
    os.makedirs("../_temp")

# aggregate_models_to_csv("../_logs/no_reflection","../_temp/aggregate.csv")
aggregate_models_to_csv("../_logs/new","../_temp/aggregate.csv")

Loading logs from directory: ../_logs/no_reflection
Skipping invalid JSON file: ../_logs/no_reflection/2025-02-07_gpt-4-0613/2025.02.07_14:12.json. Error: Expecting value: line 1 column 1 (char 0)
Total logs loaded from all directories: 3413


In [4]:
csv_file_path = "../_temp/aggregate.csv"
df_aggr = pd.read_csv(csv_file_path)
print(df_aggr.to_string(index=False))

                               model_name  total_games  black_llm_wins  white_rand_wins  draws  black_llm_win_rate  std_dev_black_llm_win_rate  moe_black_llm_win_rate  black_llm_loss_rate  std_dev_black_llm_loss_rate  moe_black_llm_loss_rate  draw_rate  std_dev_draw_rate  moe_draw_rate  black_llm_wins_percent  black_llm_draws_percent  white_rand_wins_percent  win_loss  std_dev_win_loss  moe_win_loss  win_loss_non_interrupted  std_dev_win_loss_non_interrupted  moe_win_loss_non_interrupted  game_duration  std_dev_game_duration  moe_game_duration  games_interrupted  games_interrupted_percent  std_dev_games_interrupted  moe_games_interrupted  games_not_interrupted  games_not_interrupted_percent  std_dev_games_not_interrupted  moe_games_not_interrupted  reason_too_many_wrong_actions  reason_checkmate  reason_stalemate  reason_insufficient_material  reason_seventyfive_moves  reason_fivefold_repetition  reason_max_turns  reason_unknown_issue  reason_max_moves  reason_error  llm_total_moves  a

In [6]:
# Define the columns to display
columns_to_display = [
    "model_name",
    "total_games",
    "games_interrupted_percent",
    "reason_too_many_wrong_actions",
    "reason_checkmate",
    "reason_stalemate",
    "reason_insufficient_material",
    "reason_seventyfive_moves",
    "reason_fivefold_repetition",
    "reason_max_turns",
    "reason_unknown_issue",
    "reason_max_moves",
    "reason_error"
]

# Filter the dataframe to only include the specified columns
df_filtered = df_aggr[columns_to_display]

# Display the filtered dataframe
print("Filtered Model Statistics:")
print(df_filtered.to_string(index=False))


Filtered Model Statistics:
                               model_name  total_games  games_interrupted_percent  reason_too_many_wrong_actions  reason_checkmate  reason_stalemate  reason_insufficient_material  reason_seventyfive_moves  reason_fivefold_repetition  reason_max_turns  reason_unknown_issue  reason_max_moves  reason_error
                      amazon.nova-lite-v1           42                 100.000000                             32                 0                 0                             0                         0                           0                10                     0                 0             0
                       amazon.nova-pro-v1           33                 100.000000                             33                 0                 0                             0                         0                           0                 0                     0                 0             0
anthropic.claude-3-7-sonnet-20250219-v1:0           42    

In [7]:
# Filter models with non-zero errors or unknown issues
error_columns = ["reason_error", "reason_unknown_issue"]
df_errors = df_aggr[
    (df_aggr["reason_error"] > 0) | 
    (df_aggr["reason_unknown_issue"] > 0)
]

# Select relevant columns for display
display_columns = [
    "model_name", 
    "total_games",
    "reason_error",
    "reason_unknown_issue",
    "games_interrupted_percent"
]

# Display models with errors
if len(df_errors) > 0:
    print("Models with Errors or Unknown Issues:")
    print(df_errors[display_columns].to_string(index=False))
else:
    print("No models found with errors or unknown issues.")



Models with Errors or Unknown Issues:
                         model_name  total_games  reason_error  reason_unknown_issue  games_interrupted_percent
           anthropic.claude-v3-opus           30             2                     0                  16.666667
deepseek-r1-distill-qwen-32b@q4_k_m          108             1                     0                 100.000000
        gemini-1.5-pro-preview-0409           40            18                     0                  92.500000
gemini-2.0-flash-thinking-exp-01-21           33            31                     0                 100.000000
           gemini-2.0-pro-exp-02-05           43            16                     0                  83.720930
                  gpt-4o-2024-08-06           60             1                     0                   1.666667
           llama-3.1-tulu-3-8b@q8_0           42             1                     0                 100.000000
                        llama3.1-8b           90            16    